This notebook conducts an EDA on the data provided by URW for the Data Challenge.

# Import

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

# Data Loading

In [ ]:
import constants.constants as cst
import constants.paths as pth

In [ ]:
# Dim Tables
dim_blocks = pd.read_csv(pth.DIM_BLOCKS, **cst.CSV_PARAMS)
dim_malls = pd.read_csv(pth.DIM_MALLS, **cst.CSV_PARAMS)

# Fact Tables
fact_stores = pd.read_csv(pth.FACT_STORES, **cst.CSV_PARAMS)
fact_malls = pd.read_csv(pth.FACT_MALLS, **cst.CSV_PARAMS)
fact_sri_scores = pd.read_csv(pth.FACT_SRI_SCORES, **cst.CSV_PARAMS)

# Store financials table
store_financials = pd.read_csv(pth.STORE_FINANCIALS, **cst.CSV_PARAMS)

# Cross visit table
cross_visit = pd.read_csv(pth.CROSS_VISITS, **cst.CSV_PARAMS)

# EDA

## DIM Tables

### `dim_blocks`

The `dim_blocks` table contains information about the different blocks (areas) within the malls. The columns include:
- `mall_id`: Identifier for the mall to which the block belongs.
- `block_id`: Technical identifier of the store unit.
- `block_type`: Type of the block.
- `store_code`: Code representing the store located in the block.
- `store_name`: Name of the store located in the block (available for 3 malls only).
- `retailer_code`: Retailer identifier.
- `bl1_label`: High level commercial category.
- `bl2_label`: Mid level commercial category.
- `bl3_label`: Detailed commercial category.
- `gla`: Gross Leasable Area of the block (in square meters).
- `gla_category`: Category of the Gross Leasable Area.

In [ ]:
print(dim_blocks.shape)
display(dim_blocks.head())

In [ ]:
dim_blocks.info()

In [ ]:
dim_blocks[dim_blocks.duplicated()]

### `dim_malls`

The `dim_malls` table contains information about the malls. The columns include:
- `mall_id`: Identifier for the mall.
- `mall_name`: Name of the mall (available for 3 malls only).
- `country`: Country where the mall is located.
- `opening_hour`: Opening hours of the mall.
- `closing_hour`: Closing hours of the mall.

In [ ]:
print(dim_malls.shape)
display(dim_malls.head())

In [ ]:
dim_malls.info()

In [ ]:
dim_malls[dim_malls.duplicated()]

## FACT Tables

### `fact_stores`

The `fact_stores` table contains financial and operational data for each store within the malls in long format. The columns include:
- `date`: Date of the record.
- `mall_id`: Identifier for the mall.
- `block_id`: Technical identifier of the store unit.
- `store_code`: Code representing the store located in the block.
- `retailer_id`: Retailer identifier.
- `people_in`: Number of people entering the store.
- `people_window_flow`: Number of people passing by the store window.
- `store_average_dwell_time`: Sample weighted average time spent in the store (in minutes).
- `store_median_dwell_time`: Sample weighted median time spent in the store (in minutes).
- `shopping_average_dwell_time`: Sample weighted average time spent shopping in the store (in minutes).
- `average_visited_stores`: Sample weighted average number of distinct stores visited by customers within the block.

In [ ]:
print(fact_stores.shape)
display(fact_stores.head())

In [ ]:
fact_stores.info()

In [ ]:
fact_stores.groupby("store_code")["block_id"].nunique().sort_values(ascending=False)

In [ ]:
dim_blocks[dim_blocks["store_code"] == 1054382]

Some stores can spread across multiple blocks, which may lead to duplicated `store_code` entries in the `fact_stores` table.

### `fact_malls` 

The `fact_malls` table contains aggregated financial and operational data for each mall on a daily basis. The columns include:
- `date`: Date of the record.
- `mall_id`: Identifier for the mall.
- `people_in`: Total number of people entering the mall (not unique).
- `average_dwell_time`: Average time spent in the mall (in minutes).
- `dwell_time_sample`: Number of samples used to compute the average and median dwell times.
- `median_dwell_time`: Median time spent in the mall (in minutes).

In [ ]:
print(fact_malls.shape)
display(fact_malls.head())

In [ ]:
fact_malls.info()

In [ ]:
fact_malls[fact_malls.duplicated()]

### `fact_sri_scores`

The table `fact_sri_scores` contains the SRI (Sustainability and Responsibility Index) scores for each store. The columns include:
- `store_code`: Code representing the store.
- `sri_score`: SRI score of the store (ranging from 0 to 100).

In [ ]:
print(fact_sri_scores.shape)
display(fact_sri_scores.head())

In [ ]:
fact_sri_scores.info()

In [ ]:
fact_sri_scores[fact_sri_scores.duplicated()]

## Store Financials Table

The `store_financials` table contains financial performance data for each store on a monthly basis. The columns include:
- `codestr`: Code representing the store (matches `store_code` in other tables).
- `cur_code`: Currency code for the financial data.
- `sales_r12m`: Reported or estimated sales over the last 12 months.
- `total_costs_r12m`: Total costs of occupancy over the last 12 months.

In [ ]:
print(store_financials.shape)
display(store_financials.head())

In [ ]:
store_financials.info()

In [ ]:
store_financials[store_financials.duplicated()]

## Cross Visit Table

The `cross_visit` table contains data on customer visits across different stores. The columns include:
- `store_code_1`: Code representing the first store.
- `store_code_2`: Code representing the second store.
- `total_cross_visits`: Number of customers who visited both stores.

There is one row per pair of stores.

In [ ]:
print(cross_visit.shape)
display(cross_visit.head())

In [ ]:
cross_visit.info()

In [ ]:
cross_visit[cross_visit.duplicated()]

## Mall profiles

We compare the malls on different aspects such as size, traffic, categories distribution, financial performance, and SRI scores.

### Total GLA per mall

We compute the total Gross Leasable Area (GLA) for each mall by summing the GLA of all blocks within the mall.

In [ ]:
dim_malls["total_gla"] = dim_malls["id"].map(
    dim_blocks.groupby("mall_id").agg({"gla": "sum"})["gla"]
)

dim_malls

In [ ]:
mall_name_mapping = dict(zip(dim_malls["id"], dim_malls["mall_name"], strict=False))

# Get malls with names
named_malls = {k: v for k, v in mall_name_mapping.items() if pd.notna(v)}

# Define colors for named malls
named_mall_colors = {7: "#e74c3c", 15: "#3498db", 22: "#2ecc71"}

In [ ]:
# Filter out malls with NaN total_gla
dim_malls_plot = dim_malls.dropna(subset=["total_gla"])
avg_gla = dim_malls_plot["total_gla"].mean()

# Create color array and labels
colors = [named_mall_colors.get(mall_id, "#95a5a6") for mall_id in dim_malls_plot["id"]]
labels = [named_malls.get(mall_id) for mall_id in dim_malls_plot["id"]]

# Create bar chart
fig = go.Figure()

# Add bars
fig.add_trace(
    go.Bar(
        x=dim_malls_plot["id"],
        y=dim_malls_plot["total_gla"],
        marker_color=colors,
        text=labels,
        hovertemplate="<b>%{text}</b><br>Mall ID: %{x}<br>Total GLA: %{y:.1f}k<extra></extra>",  # noqa: E501
    )
)

# Add average line
fig.add_hline(
    y=avg_gla,
    line_dash="dash",
    line_color="gray",
    annotation_text="Average GLA",
)

# Update layout
fig.update_layout(
    title="Total Gross Leasable Area (GLA) per Mall",
    xaxis_title="Mall ID",
    yaxis_title="Total GLA",
    xaxis=dict(tickmode="linear", tick0=dim_malls_plot["id"].min(), dtick=1),
    hovermode="closest",
    height=500,
)

fig.show()

### Traffic

#### Average daily foot traffic per mall

In [ ]:
daily_avg_foot_traffic = fact_malls.groupby("mall_id").agg({"people_in": "mean"})
avg_daily_foot_traffic = daily_avg_foot_traffic["people_in"].mean()

# Create color array and labels
colors = [
    named_mall_colors.get(mall_id, "#95a5a6")
    for mall_id in daily_avg_foot_traffic.index
]
labels = [named_malls.get(mall_id) for mall_id in daily_avg_foot_traffic.index]

# Create bar chart
fig = go.Figure()

# Add bars
fig.add_trace(
    go.Bar(
        x=daily_avg_foot_traffic.index,
        y=daily_avg_foot_traffic["people_in"],
        marker_color=colors,
        text=labels,
        hovertemplate="Mall ID: %{x}<br>Average People In: %{y:.1f}<extra></extra>",
    )
)

# Add average line
fig.add_hline(
    y=avg_daily_foot_traffic,
    line_dash="dash",
    line_color="gray",
    annotation_text="Average Daily Foot Traffic",
)

# Update layout
fig.update_layout(
    title="Average Daily Foot Traffic per Mall",
    xaxis_title="Mall ID",
    yaxis_title="Average People In",
    xaxis=dict(tickmode="linear", tick0=daily_avg_foot_traffic.index.min(), dtick=1),
    hovermode="closest",
    height=500,
)

fig.show()

#### Average daily dwell time per mall

In [ ]:
daily_avg_foot_traffic = fact_malls.groupby("mall_id").agg(
    {"average_dwell_time": "mean"}
)
avg_daily_foot_traffic = daily_avg_foot_traffic["average_dwell_time"].mean()

# Create color array and labels
colors = [
    named_mall_colors.get(mall_id, "#95a5a6")
    for mall_id in daily_avg_foot_traffic.index
]
labels = [named_malls.get(mall_id) for mall_id in daily_avg_foot_traffic.index]

# Create bar chart
fig = go.Figure()

# Add bars
fig.add_trace(
    go.Bar(
        x=daily_avg_foot_traffic.index,
        y=daily_avg_foot_traffic["average_dwell_time"],
        marker_color=colors,
        text=labels,
        hovertemplate="Mall ID: %{x}<br>Average Dwell Time: %{y:.1f}<extra></extra>",
    )
)

# Add average line
fig.add_hline(
    y=avg_daily_foot_traffic,
    line_dash="dash",
    line_color="gray",
    annotation_text="Average Dwell Time",
)

# Update layout
fig.update_layout(
    title="Average Daily Dwell Time per Mall",
    xaxis_title="Mall ID",
    yaxis_title="Average Dwell Time (minutes)",
    xaxis=dict(tickmode="linear", tick0=daily_avg_foot_traffic.index.min(), dtick=1),
    hovermode="closest",
    height=500,
)

fig.show()

### Category distribution per mall

#### Number of retailers per mall

In [ ]:
number_of_retailers = dim_blocks.groupby("mall_id").agg({"retailer_code": "nunique"})
number_of_retailers = number_of_retailers.rename(
    columns={"retailer_code": "nb_retailers"}
)
avg_number_of_retailers = number_of_retailers["nb_retailers"].mean()


# Create color array and labels
colors = [
    named_mall_colors.get(mall_id, "#95a5a6") for mall_id in number_of_retailers.index
]
labels = [named_malls.get(mall_id) for mall_id in number_of_retailers.index]

# Create bar chart
fig = go.Figure()

# Add bars
fig.add_trace(
    go.Bar(
        x=number_of_retailers.index,
        y=number_of_retailers["nb_retailers"],
        marker_color=colors,
        text=labels,
        hovertemplate="Mall ID: %{x}<br>Number of Retailers: %{y}<extra></extra>",
    )
)

# Add average line
fig.add_hline(
    y=avg_number_of_retailers,
    line_dash="dash",
    line_color="gray",
    annotation_text="Average Number of Retailers",
)

# Update layout
fig.update_layout(
    title="Number of Retailers per Mall",
    xaxis_title="Mall ID",
    yaxis_title="Number of Retailers",
    xaxis=dict(tickmode="linear", tick0=number_of_retailers.index.min(), dtick=1),
    hovermode="closest",
    height=500,
)

fig.show()

#### Number of categories per mall

For this, we have 3 levels of categories: bl1_label, bl2_label and bl3_label.

In [ ]:
def plot_categories_by_granularity(granularity):
    """Plot number of categories per mall for selected granularity level."""
    # Calculate number of categories
    number_of_categories = dim_blocks.groupby("mall_id").agg({granularity: "nunique"})
    number_of_categories = number_of_categories.rename(
        columns={granularity: "nb_categories"}
    )
    avg_number_of_categories = number_of_categories["nb_categories"].mean()

    # Create color array and labels
    colors = [
        named_mall_colors.get(mall_id, "#95a5a6")
        for mall_id in number_of_categories.index
    ]
    labels = [named_malls.get(mall_id) for mall_id in number_of_categories.index]

    # Create bar chart
    fig = go.Figure()

    # Add bars
    fig.add_trace(
        go.Bar(
            x=number_of_categories.index,
            y=number_of_categories["nb_categories"],
            marker_color=colors,
            text=labels,
            hovertemplate=f"Mall ID: %{{x}}<br>Number of {granularity} Categories: %{{y}}<extra></extra>",  # noqa: E501
        )
    )

    # Add average line
    fig.add_hline(
        y=avg_number_of_categories,
        line_dash="dash",
        line_color="gray",
        annotation_text=f"Average Number of {granularity} Categories",
    )

    # Update layout
    fig.update_layout(
        title=f"Number of {granularity} Categories per Mall",
        xaxis_title="Mall ID",
        yaxis_title=f"Number of {granularity} Categories",
        xaxis=dict(tickmode="linear", tick0=number_of_categories.index.min(), dtick=1),
        hovermode="closest",
        height=500,
    )

    fig.show()


# Create dropdown widget
granularity_dropdown = widgets.Dropdown(
    options=["bl1_label", "bl2_label", "bl3_label"],
    value="bl1_label",
    description="Granularity:",
    style={"description_width": "initial"},
)

# Create interactive widget
interactive_plot = widgets.interactive(
    plot_categories_by_granularity, granularity=granularity_dropdown
)
interactive_plot

## Temporal analysis

Temporal analysis of foot traffic and dwell time over all malls and per mall.

In [ ]:
fact_malls_date = fact_malls.copy()
fact_malls_date["date"] = pd.to_datetime(
    fact_malls_date["date"], format=cst.DATE_FORMAT
)
fact_malls_date.index = fact_malls_date["date"]
fact_malls_date.drop(columns=["date"], inplace=True)
fact_malls_date.sort_index(inplace=True)

In [ ]:
fact_malls_date

In [ ]:
daily_avg_foot_traffic = fact_malls_date.groupby(fact_malls_date.index).agg(
    {"people_in": "mean"}
)

# Create figure
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(
        x=daily_avg_foot_traffic.index,
        y=daily_avg_foot_traffic["people_in"],
        mode="lines",
        name="People In",
    )
)

# Update layout
fig.update_layout(
    title="Average Daily Foot Traffic Over Time - All Malls",
    xaxis_title="Date",
    yaxis_title="People In",
    hovermode="x unified",
    height=500,
)

fig.show()

In [ ]:
def plot_foot_traffic_by_mall(mall_id):
    """Plot daily foot traffic over time for a selected mall."""
    # Filter data for selected mall
    mall_data = fact_malls_date[fact_malls_date["mall_id"] == mall_id]

    # Get mall name if available
    mall_name = mall_name_mapping.get(mall_id, f"Mall {mall_id}")

    # Create figure
    fig = go.Figure()

    # Add trace
    fig.add_trace(
        go.Scatter(
            x=mall_data.index,
            y=mall_data["people_in"],
            mode="lines",
            name="People In",
            line=dict(color=named_mall_colors.get(mall_id, "#95a5a6")),
        )
    )

    # Update layout
    fig.update_layout(
        title=f"Daily Foot Traffic Over Time - {mall_name}",
        xaxis_title="Date",
        yaxis_title="People In",
        hovermode="x unified",
        height=500,
    )

    fig.show()


# Create dropdown widget with all mall IDs
mall_ids = sorted(fact_malls_date["mall_id"].unique())
mall_dropdown = widgets.Dropdown(
    options=mall_ids,
    value=mall_ids[0],
    description="Mall ID:",
    style={"description_width": "initial"},
)

# Create interactive widget
interactive_mall_plot = widgets.interactive(
    plot_foot_traffic_by_mall, mall_id=mall_dropdown
)
interactive_mall_plot

### Autocorrelation analysis

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(daily_avg_foot_traffic["people_in"])
plt.show()

In [ ]:
plot_pacf(daily_avg_foot_traffic["people_in"])
plt.show()